## Clustering

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.

In [0]:
from pyspark.sql import SparkSession 

In [0]:
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [0]:
data = spark.read.csv('dbfs:/FileStore/hack_data.csv', inferSchema=True, header=True)

In [0]:
data.limit(5).display()

Session_Connection_Time,Bytes Transferred,Kali_Trace_Used,Servers_Corrupted,Pages_Corrupted,Location,WPM_Typing_Speed,hack
8.0,391.09,1,2.96,7.0,Slovenia,72.37,1
20.0,720.99,0,3.04,9.0,British Virgin Islands,69.08,1
31.0,356.32,1,3.71,8.0,Tokelau,70.58,1
2.0,228.08,1,2.48,8.0,Bolivia,70.8,1
20.0,408.5,0,3.57,8.0,Iraq,71.28,1


In [0]:
data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- hack: integer (nullable = true)



In [0]:
# missing values
from pyspark.sql.functions import col, sum as _sum

missing_val = data.select([_sum(col(c).isNull().cast('int')).alias(c) for c in data.columns])
missing_val.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------+----------------+----+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|Location|WPM_Typing_Speed|hack|
+-----------------------+-----------------+---------------+-----------------+---------------+--------+----------------+----+
|                      0|                0|              0|                0|              0|       0|               0|   0|
+-----------------------+-----------------+---------------+-----------------+---------------+--------+----------------+----+



In [0]:
# duplicates data
dupliated = data.exceptAll(data.dropDuplicates())
dupliated.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------+----------------+----+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|Location|WPM_Typing_Speed|hack|
+-----------------------+-----------------+---------------+-----------------+---------------+--------+----------------+----+
+-----------------------+-----------------+---------------+-----------------+---------------+--------+----------------+----+



In [0]:
data.groupBy('hack').count().show()

+----+-----+
|hack|count|
+----+-----+
|   1|  167|
|   0|  167|
+----+-----+



In [0]:
data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed',
 'hack']

In [0]:
data.select(['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used',
              'Servers_Corrupted', 'Pages_Corrupted', 'WPM_Typing_Speed', 'hack']).describe().display()

summary,Session_Connection_Time,Bytes Transferred,Kali_Trace_Used,Servers_Corrupted,Pages_Corrupted,WPM_Typing_Speed,hack
count,334,334,334,334,334,334,334
mean,30.008982035928145,607.2452694610777,0.5119760479041916,5.258502994011977,10.838323353293413,57.342395209580864,0.5
stddev,14.088200614636158,286.33593163576757,0.5006065264451406,2.30190693339697,3.06352633036022,13.41106336843464,0.5007501879687625
min,1.0,10.0,0,1.0,6.0,40.0,0
max,60.0,1330.5,1,10.0,15.0,75.0,1


## Format for MLlib

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
assembler= VectorAssembler(inputCols= ['Session_Connection_Time', 'Bytes Transferred',
                                       'Kali_Trace_Used', 'Servers_Corrupted', 'Pages_Corrupted',
                                        'WPM_Typing_Speed', 'hack'], 
                           outputCol='features')

In [0]:
output = assembler.transform(data)

## Standard Scaler

In [0]:
from pyspark.ml.feature import StandardScaler

In [0]:
scaler = StandardScaler(inputCol='features', outputCol='scalerfeatures', withStd= True, withMean=False)

In [0]:
final_data = scaler.fit(output).transform(output)

In [0]:
final_data.limit(5).display()

Session_Connection_Time,Bytes Transferred,Kali_Trace_Used,Servers_Corrupted,Pages_Corrupted,Location,WPM_Typing_Speed,hack,features,scalerfeatures
8.0,391.09,1,2.96,7.0,Slovenia,72.37,1,"Map(vectorType -> dense, length -> 7, values -> List(8.0, 391.09, 1.0, 2.96, 7.0, 72.37, 1.0))","Map(vectorType -> dense, length -> 7, values -> List(0.5678510846650524, 1.3658432518957642, 1.9975768336483841, 1.2858903881191532, 2.2849485348398866, 5.396290958577967, 1.9970037436359023))"
20.0,720.99,0,3.04,9.0,British Virgin Islands,69.08,1,"Map(vectorType -> dense, length -> 7, values -> List(20.0, 720.99, 0.0, 3.04, 9.0, 69.08, 1.0))","Map(vectorType -> dense, length -> 7, values -> List(1.419627711662631, 2.517986463945197, 0.0, 1.320644182392644, 2.9377909733655687, 5.150971112595909, 1.9970037436359023))"
31.0,356.32,1,3.71,8.0,Tokelau,70.58,1,"Map(vectorType -> dense, length -> 7, values -> List(31.0, 356.32, 1.0, 3.71, 8.0, 70.58, 1.0))","Map(vectorType -> dense, length -> 7, values -> List(2.2004229530770782, 1.2444124562517545, 1.9975768336483841, 1.611707209433128, 2.6113697541027276, 5.262819066691072, 1.9970037436359023))"
2.0,228.08,1,2.48,8.0,Bolivia,70.8,1,"Map(vectorType -> dense, length -> 7, values -> List(2.0, 228.08, 1.0, 2.48, 8.0, 70.8, 1.0))","Map(vectorType -> dense, length -> 7, values -> List(0.1419627711662631, 0.7965469045293562, 1.9975768336483841, 1.0773676224782096, 2.6113697541027276, 5.279223433291696, 1.9970037436359023))"
20.0,408.5,0,3.57,8.0,Iraq,71.28,1,"Map(vectorType -> dense, length -> 7, values -> List(20.0, 408.5, 0.0, 3.57, 8.0, 71.28, 1.0))","Map(vectorType -> dense, length -> 7, values -> List(1.419627711662631, 1.4266459597520256, 0.0, 1.5508880694545193, 2.6113697541027276, 5.315014778602148, 1.9970037436359023))"


## Model Cluster KMeans

In [0]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

In [0]:
kmean2 = KMeans(featuresCol='scalerfeatures', k= 2)
kmean3 = KMeans(featuresCol='scalerfeatures', k= 3)

In [0]:
model_k2 = kmean2.fit(final_data)
model_k3 = kmean3.fit(final_data)

In [0]:
prediction_k2 = model_k2.transform(final_data)
prediction_k3 = model_k3.transform(final_data)

In [0]:
cluster_eval = ClusteringEvaluator()
silhouette_k2 = cluster_eval.evaluate(prediction_k2)
sillouette_k3 = cluster_eval.evaluate(prediction_k3)

print('with k = 2')
print(f'with Shilouette score = {silhouette_k2}')
print('*' * 100)
print('with k = 3')
print(f'with Shilouette score = {sillouette_k3}')

with k = 2
with Shilouette score = 0.6683642300696484
****************************************************************************************************
with k = 3
with Shilouette score = 0.3068099738784494


## Looking for k values, use the best score : Silhouette Score

In [0]:
silhouette_score = []
k_values = range(2,11)

for k in k_values:
  kmeans = KMeans(featuresCol='scalerfeatures').setK(k)
  model = kmeans.fit(final_data)
  predict = model.transform(final_data)

  evaluateCluster = ClusteringEvaluator()
  silhouette = evaluateCluster.evaluate(predict)
  silhouette_score.append(silhouette)
  print(f'K : {k}, Shilhouette score : {silhouette}')
  print('*' * 100)

K : 2, Shilhouette score : 0.6683642300696484
****************************************************************************************************
K : 3, Shilhouette score : 0.3068099738784494
****************************************************************************************************
K : 4, Shilhouette score : -0.04792790081154923
****************************************************************************************************
K : 5, Shilhouette score : -0.1021952281772995
****************************************************************************************************
K : 6, Shilhouette score : -0.14728076778236637
****************************************************************************************************
K : 7, Shilhouette score : -0.16032261356804753
****************************************************************************************************
K : 8, Shilhouette score : -0.16306777526764035
***************************************************************

In [0]:
model_k3.transform(final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   83|
|         2|   84|
|         0|  167|
+----------+-----+



In [0]:
model_k2.transform(final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



Bingo! It was 2 hackers, in fact, our clustering algorithm created two equally sized clusters with K=2, no way that is a coincidence!
## Great Job!